### Please run with Google Colab with Good GPU
<a href="https://colab.research.google.com/github/Ichikawa-Satoshi/SI-Org-chart/blob/main/make_data/steps/deeplearning_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Detectron2 has not released pre-built binaries for the latest pytorch (https://github.com/facebookresearch/detectron2/issues/4053)
# so we install from source instead. This takes a few minutes.
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-zctcpv_s
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-zctcpv_s
  Resolved https://github.com/facebookresearch/detectron2.git to commit 9604f5995cc628619f0e4fd913453b4d7d61db3f
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 82.7 MB/s eta 0:00:00
  Created wheel for detectron2: filename=detectron2-0.6-cp3

In [2]:
import cv2 as cv2
import json
import os
from google.colab import drive
drive.mount('/content/drive')

# import some common detectron2 utilities
import matplotlib.pyplot as plt
import detectron2
from tqdm import tqdm
from detectron2.utils.logger import setup_logger
setup_logger()
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.visualizer import Visualizer, ColorMode

Mounted at /content/drive


In [3]:
path = "/content/drive/MyDrive/SI-Org-Chart/data/Org_chart/learning"
path_train = path + "/train"
path_coco = path + "/Organization_annotation.json"
path_data = "/content/drive/MyDrive/SI-Org-Chart/data/Org_chart/cropped"
# set train data
register_coco_instances("org_chart_train", {}, path_coco, path_train)

# setting for using the model
cfg = get_cfg() # initialize
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("org_chart_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 1
cfg.SOLVER.BASE_LR = 0.0004
cfg.SOLVER.MAX_ITER = (500)
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = (128)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4

# train
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True) # for output
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[03/10 02:47:59 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

model_final_f10217.pkl: 178MB [00:01, 177MB/s]                           
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.predictor.{bias, weight}


[03/10 02:48:02 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.11/dist-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[03/10 02:48:21 d2.utils.events]:  eta: 0:03:34  iter: 19  total_loss: 6.119  loss_cls: 1.433  loss_box_reg: 0.5506  loss_mask: 0.6874  loss_rpn_cls: 3.154  loss_rpn_loc: 0.3641    time: 0.6163  last_time: 0.7759  data_time: 0.3638  last_data_time: 0.5663   lr: 1.5585e-05  max_mem: 1308M
[03/10 02:48:38 d2.utils.events]:  eta: 0:03:08  iter: 39  total_loss: 3.273  loss_cls: 1.22  loss_box_reg: 0.6443  loss_mask: 0.6412  loss_rpn_cls: 0.3464  loss_rpn_loc: 0.29    time: 0.6236  last_time: 0.5341  data_time: 0.4431  last_data_time: 0.3655   lr: 3.1569e-05  max_mem: 1454M
[03/10 02:48:48 d2.utils.events]:  eta: 0:03:00  iter: 59  total_loss: 2.495  loss_cls: 0.9057  loss_box_reg: 0.6978  loss_mask: 0.5624  loss_rpn_cls: 0.1005  loss_rpn_loc: 0.2485    time: 0.5722  last_time: 0.2381  data_time: 0.2528  last_data_time: 0.0776   lr: 4.7553e-05  max_mem: 1484M
[03/10 02:48:58 d2.utils.events]:  eta: 0:02:59  iter: 79  total_loss: 2.197  loss_cls: 0.6479  loss_box_reg: 0.6626  loss_mask: 0.46

In [4]:
from PIL import Image
from concurrent.futures import ThreadPoolExecutor

def get_image_size(image_path):
    try:
        with Image.open(image_path) as img:
            return img.size  # (width, height)
    except Exception as e:
        print(f"{image_path}: {e}")
        return None
def get_test_dicts(img_dir):
    img_files = [os.path.join(img_dir, f) for f in os.listdir(img_dir) if f.lower().endswith(('.jpg', '.png'))]
    dataset_dicts = []
    with ThreadPoolExecutor() as executor:
        sizes = list(executor.map(get_image_size, img_files))
    for idx, (img_file, size) in enumerate(zip(img_files, sizes)):
        if size is not None:
            width, height = size
            record = {
                "file_name": img_file,
                "image_id": idx,
                "width": width,
                "height": height
            }
            dataset_dicts.append(record)
    return dataset_dicts

# change the test data form for dectron2
DatasetCatalog.register("orgs", lambda: get_test_dicts(path_data))
MetadataCatalog.get("orgs").set(thing_classes=["department"])

In [5]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # load trained weights
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8  # score
cfg.DATASETS.TEST = ("orgs", )  # set the test data to the model

# detect departments
predictor = DefaultPredictor(cfg)
metadata = MetadataCatalog.get("orgs")
dataset_dicts = DatasetCatalog.get("orgs")
output_path = "/content/drive/MyDrive/SI-Org-Chart/data/Org_chart/output"
for d in tqdm(dataset_dicts):
    img = cv2.imread(d["file_name"])
    outputs = predictor(img)
    json_output = {
    "file_name": d["file_name"],
    "pred_boxes": outputs["instances"].pred_boxes.tensor.cpu().numpy().tolist(),
    "scores": outputs["instances"].scores.cpu().numpy().tolist(),
    "pred_classes": outputs["instances"].pred_classes.cpu().numpy().tolist(),
    "num_department": len(outputs["instances"].pred_boxes.tensor.cpu().numpy().tolist())
    }
    # save JSON
    base_name = os.path.basename(d["file_name"])
    json_name = base_name.replace(".png", ".json")
    json_path = os.path.join(output_path, json_name)
    with open(json_path, "w") as f:
        json.dump(json_output, f)

[03/10 02:51:03 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./output/model_final.pth ...


/usr/local/lib/python3.11/dist-packages/fvcore/common/checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=torch.device("cpu"))

Traceback (most recent call last):
  File "<ipython-input-4-d07fde597417>", line 15, in get_test_dicts
    sizes = list(executor.map(get_image_size, img_files))
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/concurrent/futures/_base.py", line 619, in result_iterator
    yield _result_or_cancel(fs.pop())
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/concurrent/futures/_base.py", line 317, in _result_or_cancel
    return fut.result(timeout)
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/concurrent/futures/_base.py", line 451, in result
    self._condition.wait(timeout)
  File "/usr/lib/python3.11/threading.py", line 327, in wait
    waiter.acquire()
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self

TypeError: object of type 'NoneType' has no len()